# Some examples of substitution cryptos

First, some imports.

In [1]:
import nltk
from nltk.corpus import gutenberg
from ngram import NGramModel
import numpy as np
import os
from tqdm import tqdm
from multiprocessing import Pool

Project gutenberg has some books that can easily be downloaded and used as reference data.

In [2]:
nltk.download('gutenberg')

print("Available books:", gutenberg.fileids())

Available books: ['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


[nltk_data] Downloading package gutenberg to
[nltk_data]     /home/fredrik/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


## Lexicon

A nice list of english words that could be used for comparing decoded data to real words.

In [3]:
fileids = gutenberg.fileids()
if 'bible-kjv.txt' in fileids:
    fileids.remove('bible-kjv.txt')
all_words = set()
for fileid in tqdm(fileids):
    all_words.update(gutenberg.words(fileid))
lexicon = set([e.lower() for e in all_words if e.isalpha()])

print("We have %i english (non-lemmatized) words in the extracted lexicon" % len(lexicon))

100%|██████████| 17/17 [00:07<00:00,  2.37it/s]

We have 36160 english (non-lemmatized) words in the extracted lexicon


Some examples from the lexicon

In [4]:
from random import choices
print(choices(list(lexicon), k=20))

['oarsmen', 'instigated', 'wig', 'architectural', 'soldier', 'conjured', 'cargoes', 'confounds', 'shellfish', 'effeminacy', 'daylight', 'cosa', 'promenade', 'bowels', 'insouciance', 'cannikin', 'envelopes', 'vapoured', 'questioning', 'confines']


## n-gram models
Let's clean up some Austen books as training data for the character models

In [5]:
print(gutenberg.raw('austen-emma.txt')[:600])

[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.  Her mother
had died too long ago for her to have more than an indistinct
remembrance of her caresses; and her place had b


In [6]:
def generate_alphabet(alpha, omega):
    """Set of the english alphabet"""
    return set([chr(i) for i in range(ord(alpha), ord(omega)+1)]) 

def clean_text(text, allowed):
    ret = text.lower()
    strip = set(ret).difference(allowed)
    if " " in allowed:
        for s in strip:
            if s in ['\n', '\t']:
                ret = ret.replace(s, " ")
            else:
                ret = ret.replace(s, "")
        for i in range(5): # HACK
            ret = ret.replace("  ", " ")
    else:
        for s in strip:
            ret = ret.replace(s, "")
    return ret

alphabet = generate_alphabet('a', 'z') # Set of the english alphabet
allowed_characters = set([' ', '.'])
allowed_characters.update(alphabet)

cleaned_text = clean_text(gutenberg.raw('austen-emma.txt'), allowed_characters)
print(cleaned_text[:800])

emma by jane austen volume i chapter i emma woodhouse handsome clever and rich with a comfortable home and happy disposition seemed to unite some of the best blessings of existence and had lived nearly twentyone years in the world with very little to distress or vex her. she was the youngest of the two daughters of a most affectionate indulgent father and had in consequence of her sisters marriage been mistress of his house from a very early period. her mother had died too long ago for her to have more than an indistinct remembrance of her caresses and her place had been supplied by an excellent woman as governess who had fallen little short of a mother in affection. sixteen years had miss taylor been in mr. woodhouses family less as a governess than a friend very fond of both daughters bu


With the data cleaned, we are ready to create some character ngram models. 

In [7]:
books = list()
for fileid in tqdm(fileids):
    books.append(clean_text(gutenberg.raw(fileid), allowed_characters))

100%|██████████| 17/17 [00:00<00:00, 17.45it/s]


In [8]:
n_model = 5

from sklearn.model_selection import ParameterGrid
grid = ParameterGrid({'data': books,
                      'order': list(range(1, n_model+1))})

def make_model(grid_point):
    chars = list(grid_point['data'])
    return NGramModel(chars, grid_point['order'])

ngram_models = [None]*(n_model+1)
with Pool(processes=os.cpu_count()) as process_pool:
    for model in tqdm(process_pool.imap_unordered(make_model, grid),
                      total=len(grid), desc="Making n-gram models"):
        if ngram_models[model.order_] is None:
            ngram_models[model.order_] = model
        else:
            ngram_models[model.order_] = ngram_models[model.order_].union(model)

for order in range(1, n_model):
    print(ngram_models[order])

Making n-gram models: 100%|██████████| 85/85 [01:14<00:00,  1.99s/it]


1-gram model with 28 unique keys
2-gram model with 690 unique keys
3-gram model with 8517 unique keys
4-gram model with 50795 unique keys


In [9]:
for order in range(1, n_model+1):
    print("%i-gram model: %s" % (order, "".join(ngram_models[order].predict_sequence(80))))
    print()

1-gram model: ut parhtroismu owfouarpww oeiao  o  e dt cidrnlmyaco e eeredtrmenln sfoaessliybs

2-gram model: an t ror an. sphertevericly d onor fof antay gen dse s kish t whas. ticanole s f

3-gram model: mbut once of has proby st fat on har lied le and frove sly th winfarm vould th b

4-gram model: hat and a forwarneral and almsmalemed ther on quire the sticatic that reput tal 

5-gram model: smith he jem. the mosco my hope it but so. the similar fate better. volted from 



We can now find the most common characters in this english text and their probabilities (from relative frequencies).

In [10]:
order = 3

from ngram import ordered_ngrams
for key, prob in list(ordered_ngrams(ngram_models[order]))[:10]:
    print("%s - %.5f" % (key, prob))

(' ', 't', 'h') - 0.01895
('t', 'h', 'e') - 0.01476
('h', 'e', ' ') - 0.01313
('n', 'd', ' ') - 0.00780
(' ', 'a', 'n') - 0.00755
('a', 'n', 'd') - 0.00718
('e', 'd', ' ') - 0.00620
('i', 'n', 'g') - 0.00588
(' ', 't', 'o') - 0.00571
('e', 'r', ' ') - 0.00565


## Caesar substitution crypto
Maybe the most basic substitution crypto, based on charcter rotations.

In [11]:
def caesar_encryption(word, offset=3):
    enc = [chr(ord(char)+offset-len(alphabet)) if (ord(char)+offset)>ord('z') else chr(ord(char)+offset)
           for char in word.lower() if char in alphabet]
    return "".join(enc).upper()

import random
offset = random.randint(1, len(alphabet)-1)

print(caesar_encryption("Et tu brute", offset))

YNNOVLONY


Now for finding the key to an unknown cryptogram (assuming it's a caesar crypto).

In [12]:
message_to_alesia = "YHUFLQJHWRULABRXUPRWKHUZDVDKDPVWHUDQGBRXUIDWKHUVPHOOVRIHOGHUEHUULHV"

In [13]:
caesar_model = NGramModel(list(message_to_alesia), 1)
for unigram, prob in list(ordered_ngrams(caesar_model)):
    print("%s - %.5f" % (unigram, prob))

('H',) - 0.14925
('U',) - 0.14925
('V',) - 0.07463
('D',) - 0.07463
('R',) - 0.07463
('W',) - 0.05970
('L',) - 0.04478
('K',) - 0.04478
('P',) - 0.04478
('O',) - 0.04478
('B',) - 0.02985
('I',) - 0.02985
('G',) - 0.02985
('X',) - 0.02985
('Q',) - 0.02985
('F',) - 0.01493
('E',) - 0.01493
('J',) - 0.01493
('A',) - 0.01493
('Z',) - 0.01493
('Y',) - 0.01493


In [14]:
def caesar_decode(text, offset):
    ret = str()
    for i in range(len(text)):
        c = ord(text[i]) - offset
        if c > ord('Z'):
            c -= len(alphabet)
        if c < ord('A'):
            c += len(alphabet)
        ret += chr(c)
    return ret

In [15]:
n_key = 3
message_from_caesar = caesar_decode(message_to_alesia, n_key)
print(message_from_caesar.capitalize())

Vercingetorixyourmotherwasahamsterandyourfathersmellsofelderberries


Something we can do to solve this, but would be harder for the romans, is to let a computer brute force this. However short the message we could try every solution and compare to a dictionary.

In [16]:
for n in range(1, len(alphabet)):
    print(n, caesar_decode(message_to_alesia, n).lower())

1 xgtekpigvqtkzaqwtoqvjgtycucjcouvgtcpfaqwthcvjgtuognnuqhgnfgtdgttkgu
2 wfsdjohfupsjyzpvsnpuifsxbtbibntufsboezpvsgbuifstnfmmtpgfmefscfssjft
3 vercingetorixyourmotherwasahamsterandyourfathersmellsofelderberries
4 udqbhmfdsnqhwxntqlnsgdqvzrzgzlrsdqzmcxntqezsgdqrldkkrnedkcdqadqqhdr
5 tcpaglecrmpgvwmspkmrfcpuyqyfykqrcpylbwmspdyrfcpqkcjjqmdcjbcpzcppgcq
6 sbozfkdbqlofuvlrojlqebotxpxexjpqboxkavlrocxqebopjbiiplcbiaboyboofbp
7 ranyejcapknetukqnikpdanswowdwiopanwjzukqnbwpdanoiahhokbahzanxanneao
8 qzmxdibzojmdstjpmhjoczmrvnvcvhnozmviytjpmavoczmnhzggnjazgyzmwzmmdzn
9 pylwchaynilcrsiolginbylqumubugmnyluhxsiolzunbylmgyffmizyfxylvyllcym
10 oxkvbgzxmhkbqrhnkfhmaxkptltatflmxktgwrhnkytmaxklfxeelhyxewxkuxkkbxl
11 nwjuafywlgjapqgmjeglzwjoskszseklwjsfvqgmjxslzwjkewddkgxwdvwjtwjjawk
12 mvitzexvkfizopflidfkyvinrjryrdjkvireupfliwrkyvijdvccjfwvcuvisviizvj
13 luhsydwujehynoekhcejxuhmqiqxqcijuhqdtoekhvqjxuhicubbievubtuhruhhyui
14 ktgrxcvtidgxmndjgbdiwtglphpwpbhitgpcsndjgupiwtghbtaahdutastgqtggxth
15 jsfqwbushcfw

## One-to-one substitution crypto

In [17]:
text = """Whats this then Romanes eunt domus People called Romanes they go the house It says Romans go home"""
#def encrypt_substitution(text):
text = text.lower()
unenc = list(set(text))
symbols = [c.upper() for c in alphabet]
symbols.extend(list("0123456789"))
s = list(symbols)
random.shuffle(s)
enc = s[:len(unenc)]

key = dict()
for a, b in zip(unenc, enc):
    key[a] = b

def substitute(text, encryption_key, replace="-"):
    ret = str()
    for c in text:
        if c in encryption_key.keys():
            ret += encryption_key[c]
        else:
            if replace is not None:
                ret += replace
            else:
                ret += c
    return ret

enc_text = substitute(text, key)
print(enc_text)

O9WLCNL9JCNL9TANFE1WATCNTDALNBE1DCNZTEZ5TNVW55TBNFE1WATCNL9T3N8ENL9TN9EDCTNJLNCW3CNFE1WACN8EN9E1T


In [18]:
key

{'c': 'V',
 'r': 'F',
 ' ': 'N',
 't': 'L',
 'g': '8',
 'e': 'T',
 'l': '5',
 'd': 'B',
 'u': 'D',
 'h': '9',
 's': 'C',
 'o': 'E',
 'y': '3',
 'w': 'O',
 'p': 'Z',
 'i': 'J',
 'm': '1',
 'a': 'W',
 'n': 'A'}

In [19]:
reverse_key = dict()
for a, b in [(k, key[k]) for k in key.keys()]:
    reverse_key[b] = a
reverse_key

{'V': 'c',
 'F': 'r',
 'N': ' ',
 'L': 't',
 '8': 'g',
 'T': 'e',
 '5': 'l',
 'B': 'd',
 'D': 'u',
 '9': 'h',
 'C': 's',
 'E': 'o',
 '3': 'y',
 'O': 'w',
 'Z': 'p',
 'J': 'i',
 '1': 'm',
 'W': 'a',
 'A': 'n'}

In [20]:
print(substitute(enc_text, reverse_key))

whats this then romanes eunt domus people called romanes they go the house it says romans go home


## Real problem

In [21]:
intercepted_in_jerusalem = """GPMCPIMEHMIRJXDCRMIRPMDH1PJCJXDH1MRP7IEHCMT4TIPLMRPJPMDQMIRJXDCRMIXMIRPML7EHM7D1EPH2PM2R7LWPJMRPJPM7H1MQE07IPTMGESPTMWP1JXXLMETMRPJPMR7KEHCMCJ7WWP1MRETMGESPMGPMEHSXJLMQE07IPMIR7IMTRPMETMEHMXDJM2DTIX14M7H1MSXJIRGEIRMETTDPMXDJM1PL7H1TMIRP4KPMW0P1MDTMGREIPMIRPMW7TI7J1TMIRP4KPMI75PHMPKPJ4IREHCMGPMR71MHXIMBDTIMSJXLMDTMSJXLMXDJMS7IRPJTM7H1MSJXLMXDJMS7IRPJTMS7IRPJT"""
print(intercepted_in_jerusalem)

GPMCPIMEHMIRJXDCRMIRPMDH1PJCJXDH1MRP7IEHCMT4TIPLMRPJPMDQMIRJXDCRMIXMIRPML7EHM7D1EPH2PM2R7LWPJMRPJPM7H1MQE07IPTMGESPTMWP1JXXLMETMRPJPMR7KEHCMCJ7WWP1MRETMGESPMGPMEHSXJLMQE07IPMIR7IMTRPMETMEHMXDJM2DTIX14M7H1MSXJIRGEIRMETTDPMXDJM1PL7H1TMIRP4KPMW0P1MDTMGREIPMIRPMW7TI7J1TMIRP4KPMI75PHMPKPJ4IREHCMGPMR71MHXIMBDTIMSJXLMDTMSJXLMXDJMS7IRPJTM7H1MSJXLMXDJMS7IRPJTMS7IRPJT


### Unigram

In [22]:
from ngram import ordered_ngrams
for unigram, prob in list(ordered_ngrams(ngram_models[1]))[:10]:
    print("%s - %.5f" % (unigram, prob))

(' ',) - 0.18567
('e',) - 0.10031
('t',) - 0.07226
('a',) - 0.06456
('o',) - 0.06171
('n',) - 0.05531
('i',) - 0.05443
('h',) - 0.05213
('s',) - 0.05199
('r',) - 0.04713


In [23]:
m1 = NGramModel(list(intercepted_in_jerusalem), 1)
for unigram, prob in list(ordered_ngrams(m1))[:10]:
    print("%s - %.5f" % (unigram, prob))

('M',) - 0.17500
('P',) - 0.11111
('I',) - 0.07500
('R',) - 0.07222
('J',) - 0.06667
('T',) - 0.05556
('7',) - 0.05556
('E',) - 0.05000
('X',) - 0.04722
('H',) - 0.04444


In [24]:
key = dict()
key['M'] = ' '
key['P'] = 'e'

print(substitute(intercepted_in_jerusalem, key, replace=None))

Ge CeI EH IRJXDCR IRe DH1eJCJXDH1 Re7IEHC T4TIeL ReJe DQ IRJXDCR IX IRe L7EH 7D1EeH2e 2R7LWeJ ReJe 7H1 QE07IeT GESeT We1JXXL ET ReJe R7KEHC CJ7WWe1 RET GESe Ge EHSXJL QE07Ie IR7I TRe ET EH XDJ 2DTIX14 7H1 SXJIRGEIR ETTDe XDJ 1eL7H1T IRe4Ke W0e1 DT GREIe IRe W7TI7J1T IRe4Ke I75eH eKeJ4IREHC Ge R71 HXI BDTI SJXL DT SJXL XDJ S7IReJT 7H1 SJXL XDJ S7IReJT S7IReJT


### Bigram

In [25]:
m2 = NGramModel(list(intercepted_in_jerusalem), 2)
for unigram, prob in list(ordered_ngrams(m2))[:10]:
    print("%s - %.5f" % (unigram, prob))

('P', 'M') - 0.04735
('I', 'R') - 0.03900
('R', 'P') - 0.03621
('T', 'M') - 0.03064
('M', 'I') - 0.02786
('P', 'J') - 0.02507
('E', 'H') - 0.01950
('M', 'R') - 0.01950
('J', 'X') - 0.01950
('X', 'D') - 0.01950


In [26]:
for bigram, prob in list(ordered_ngrams(ngram_models[2]))[:10]:
    print("%s - %.5f" % (bigram, prob))

('e', ' ') - 0.03517
(' ', 't') - 0.02770
('t', 'h') - 0.02415
('h', 'e') - 0.02284
('d', ' ') - 0.02077
(' ', 'a') - 0.02064
('s', ' ') - 0.01926
('t', ' ') - 0.01918
(' ', 's') - 0.01478
('i', 'n') - 0.01457


### Match words

In [27]:
result = list()
for w in lexicon:
    if len(w) == 2 and w[1]=='e':
        result.append(w)
print(len(result), "words found")
print(result)

18 words found
['we', 'te', 'he', 'ye', 'le', 'ne', 'me', 'se', 'ge', 'be', 'ee', 've', 're', 'de', 'fe', 'pe', 'ze', 'ue']


## MCMC



In [28]:
encrypted_message = intercepted_in_jerusalem
encrypted_symbols = np.unique(list(encrypted_message))
random.shuffle(encrypted_symbols)
encrypted_symbols = "".join(encrypted_symbols)
print("Encrypted symbols:", encrypted_symbols)

decrypted_symbols = [k[0] for k in ngram_models[1].keys()]
random.shuffle(decrypted_symbols)
decrypted_symbols = "".join(decrypted_symbols)
print("Decrypted symbols:", decrypted_symbols)

assert len(set(decrypted_symbols).intersection(set(encrypted_symbols)))==0

Encrypted symbols: XER75SMWIJ01Q2H4TGPDCBKL
Decrypted symbols: gbtuidsqpvyhmncafljew xroz.k


In [29]:
n_order = 2
reference_model = ngram_models[n_order]
print("Reference model:", reference_model)

# Define a substitution function
import copy
def decode(encrypted_message, encrypted_symbols, decrypted_symbols):
    ret = copy.copy(encrypted_message)
    for e, d in zip(list(encrypted_symbols), list(decrypted_symbols)):
        ret = ret.replace(e, d)
    return ret

# Define a probability function
from ngram import grouping_tokenizer
M = dict()
for ngram, prob in list(ordered_ngrams(reference_model)):
    M[ngram] = np.log(prob)
M_min = np.min(list(M.values()))-2

def P(decrypted_message):
    s = 0
    for ngram in grouping_tokenizer(n_order).tokenize(list(decrypted_message)):
        if ngram in M:
            s += M[ngram]
        else:
            s += M_min
    return s

decrypted_message = decode(encrypted_message, encrypted_symbols, decrypted_symbols)
p = P(decrypted_message)
print("log P:", p)

Reference model: 2-gram model with 690 unique keys
log P: -3779.7640096310993


In [30]:
for iteration in range(5000):
    # Pick symbols to switch
    p1 = np.random.randint(len(decrypted_symbols))
    p2 = np.random.randint(len(decrypted_symbols))
    while p1 == p2:
        p2 = np.random.randint(len(decrypted_symbols))
    new_decrypted_symbols = list(decrypted_symbols)
    new_decrypted_symbols[p2] = decrypted_symbols[p1]
    new_decrypted_symbols[p1] = decrypted_symbols[p2]
    new_decrypted_symbols = "".join(new_decrypted_symbols)

    decrypted_message = decode(encrypted_message, encrypted_symbols, new_decrypted_symbols)
#    model = NGramModel(decrypted_message, n_order)

#    decrypted_vector = model.vectorize(codebook=reference_model.keys())
    
    p_star = P(decrypted_message)

    #print(div(reference_vector, decrypted_vector))
    if np.random.uniform() < np.exp(p_star - p):
        decrypted_symbols = new_decrypted_symbols
        p = p_star
    if iteration % 250 == 0:
        print("iter %i, logP=%.1f : %s" % (iteration, p, decrypted_message[:60]))

iter 0, logP=-3779.8 : ljswjpsbqsptvgewtsptjseqhjvwvgeqhstjupbqwsfafpjrstjvjsemsptv
iter 250, logP=-2152.2 : me geb on bradugr bre unheagadunh reibong flfbe. reae uc bra
iter 500, logP=-1953.6 : ce get on tharugh the undeagarund heitong slste. heae uy tha
iter 750, logP=-1940.7 : fe get on tharugh the undeagarund heitong s.stel heae uy tha
iter 1000, logP=-1893.0 : we get oq thraugh the uqdergrauqd heitoqg s.stel here uy thr
iter 1250, logP=-1893.0 : we get on txraugx txe undergraund xeitong s.stel xere uy txr
iter 1500, logP=-1893.0 : we get oj thraugh the ujdergraujd heitojg s.stel here uy thr
iter 1750, logP=-1867.1 : we ge. an .hrough .he underground hei.ang sts.el here uy .hr
iter 2000, logP=-1854.5 : we get in thbough the undebgbound heating system hebe u. thb
iter 2250, logP=-1849.3 : we get in through the unjergrounj heating s.stem here uy thr
iter 2500, logP=-1850.6 : we get in tbrougb tbe underground beating s.stef bere uy tbr
iter 2750, logP=-1846.3 : we .et in throu.h the u

In [31]:
print(encrypted_symbols, "=>", decrypted_symbols)
print()
print(encrypted_message)
print(decrypted_message)

XER75SMWIJ01Q2H4TGPDCBKL => oihakm ltrvdpcn.sweugbyfjzxq

GPMCPIMEHMIRJXDCRMIRPMDH1PJCJXDH1MRP7IEHCMT4TIPLMRPJPMDQMIRJXDCRMIXMIRPML7EHM7D1EPH2PM2R7LWPJMRPJPM7H1MQE07IPTMGESPTMWP1JXXLMETMRPJPMR7KEHCMCJ7WWP1MRETMGESPMGPMEHSXJLMQE07IPMIR7IMTRPMETMEHMXDJM2DTIX14M7H1MSXJIRGEIRMETTDPMXDJM1PL7H1TMIRP4KPMW0P1MDTMGREIPMIRPMW7TI7J1TMIRP4KPMI75PHMPKPJ4IREHCMGPMR71MHXIMBDTIMSJXLMDTMSJXLMXDJMS7IRPJTM7H1MSJXLMXDJMS7IRPJTMS7IRPJT
wz gzt in through thz undzrground hzating s.stzf hzrz up through to thz fain audizncz chaflzr hzrz and pivatzs wimzs lzdroof is hzrz haying grallzd his wimz wz inmorf pivatz that shz is in our custod. and morthwith issuz our dzfands thz.yz lvzd us whitz thz lastards thz.yz takzn zyzr.thing wz had not bust mrof us mrof our mathzrs and mrof our mathzrs mathzrs
